In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import AdamW

import numpy as np
import pandas as pd
from tqdm import tqdm
import re

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

SEED = 183441
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
train_df = pd.read_csv('/kaggle/input/nmt-23-languages/cleaned-train.csv')
test_df  = pd.read_csv('/kaggle/input/nmt-23-languages/cleaned-test.csv')
train_df.drop(columns=['Unnamed: 0'], inplace=True)
test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
sent1s = set(train_df[train_df['Lang1'] == 'hin']['Sentence_Lang1'].tolist())
sent2s = set(train_df[train_df['Lang2'] == 'hin']['Sentence_Lang2'].tolist())
hin_sentences = sent1s.union(sent2s)
sentences = list(hin_sentences)

In [5]:
def remove_special_chars_translate(input_string):
    input_string = input_string.replace('\n', '')
    input_string = re.sub(r'\d+', '', input_string)
    special_chars = "!@#$%^&*()[]{};:,./<>?\|।'`~-=+"
    translation_table = str.maketrans('', '', special_chars)
    return input_string.translate(translation_table)

In [6]:
for i in tqdm(range(len(sentences))):
    sentences[i] = remove_special_chars_translate(sentences[i])

100%|██████████| 72985/72985 [00:01<00:00, 68050.15it/s]


In [7]:
words = set()

for i in tqdm(range(len(sentences))):
    word = sentences[i].split()
    words = words.union(set(word))

words = list(words)
n_vocab = len(words)

100%|██████████| 72985/72985 [01:17<00:00, 943.97it/s]


In [8]:
word_idx = {}

for i, word in enumerate(words):
    word_idx[word] = i
word_idx['bos'] = n_vocab
word_idx['eos'] = n_vocab + 1

idx_word = {v:k for k,v in word_idx.items()}

In [9]:
cbow_data = []

for sentence in tqdm(sentences):
    splits = sentence.split()
    for i in range(0, len(splits)):
        target = words.index(splits[i])
        context = []
        j = i - 4
        k = i + 1
        while j < 0:
            context.append(n_vocab)
            j += 1
        while j < i:
            context.append(word_idx[splits[j]])
            j += 1
        while k <= i + 4:
            if k < len(splits) - 1:
                context.append(word_idx[splits[k]])
            else:
                context.append(n_vocab+1)
            k += 1
        cbow_data.append([context, target])

100%|██████████| 72985/72985 [12:20<00:00, 98.54it/s]


In [10]:
class CBOWDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = torch.tensor(self.data[idx][0])
        target = torch.tensor(self.data[idx][1])
        return {'context' : context, 'target' : target}
        

In [11]:
class  CBOWModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOWModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size)
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        vector = self.embedding(x)
        vector = torch.mean(vector, dim=1)
        return self.linear(vector)

    def get_embedding():
        return self.embedding

In [12]:
dataset = CBOWDataset(cbow_data)

In [13]:
cbow_data[0]

context = ""
for token in cbow_data[0][0]:
    context += idx_word[token] + " "
print(context)
print(idx_word[cbow_data[0][1]])

bos bos bos bos वह धनकोर बा से 
फिर


In [14]:
class EarlyStopping:
    def __init__(self, tolerance = 10, min_delta = 0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.min_loss  = float('inf')
        self.counter   = 0
        self.early_stop = False
        self.model_dict = None
        self.optim_dict = None
        self.epoch = -1
        
    def __call__(self, val_loss, model_dict, optim_dict, epoch):
        if np.isnan(val_loss):
            self.early_stop = True
        elif (self.min_loss - val_loss) > self.min_delta:
            self.counter = 0
            self.min_loss = val_loss
            self.model_dict = model_dict
            self.optim_dict = optim_dict
            self.epoch = epoch
        else:
            self.counter += 1
            if self.counter > self.tolerance:
                self.early_stop = True

In [15]:
n_epochs = 55
lr = 5e-4
batch_size = 64

models = []
for dim in [50, 100, 300]:
    models.append(CBOWModel(n_vocab+2, dim).to(device))
criterion = nn.CrossEntropyLoss()
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [16]:
embed_dims = [50, 100, 300]
for j in range(len(embed_dims)):
    model = models[j].to(device)
    optimizer = AdamW(model.parameters(), lr = lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=0)
    es = EarlyStopping()
    for i in range(n_epochs):
        train_loss = 0
        model.train()
        for batch in tqdm(dataloader):
            context = batch['context'].to(device)
            target = batch['target'].to(device)
            outputs = model(context)
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        print(f"Epoch {i+1}, Loss is : {train_loss/len(dataloader)}")
        es(train_loss, model.state_dict(), optimizer.state_dict(), i)
        if es.early_stop:
            break
    model.load_state_dict(es.model_dict)
    torch.save(model.state_dict(), f'CBOW_{embed_dims[j]}.pth')

100%|██████████| 20868/20868 [01:17<00:00, 270.22it/s]


Epoch 1, Loss is : 7.394787540992421


100%|██████████| 20868/20868 [01:16<00:00, 272.39it/s]


Epoch 2, Loss is : 6.893659031274608


100%|██████████| 20868/20868 [01:16<00:00, 273.20it/s]


...

Epoch 53, Loss is : 3.8736687557482963


100%|██████████| 20868/20868 [04:04<00:00, 85.49it/s]


Epoch 54, Loss is : 3.873138574398133


100%|██████████| 20868/20868 [04:04<00:00, 85.45it/s]


Epoch 55, Loss is : 3.871746539943904
